In [ ]:
5.
- staticmethod, classmethod
- descriptors
- custom containers
- subclasses and MRO
- metaclasses

## staticmethod, classmethod
Methods without `self`

In [68]:
import json

class MessageBox:
    def __init__(self, title_text, body_text, dismiss_button="Okay"):
        self.title = title_text
        self.body = body_text
        self.buttons = [dismiss_button]

    def __repr__(self):
        return f'<{self.__class__.__name__} {self.title}>'
    
    @classmethod
    def from_json(cls, jtext):
        if not cls.valid_json(jtext):
            raise ValueError('Invalid JSON for this class')
        return cls(**json.loads(jtext))
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button'}
        return required <= set(data) <= everything

m = MessageBox.from_json('''
{
  "title_text": "Alert",
  "body_text": "Something bad happened"
}''')
m

<MessageBox Alert>

In [69]:
m.buttons

['Okay']

In [70]:
MessageBox.valid_json('42')

False

In [64]:
MessageBox.valid_json('{"title_text":"a","body_text":"b"}')

True

In [71]:
MessageBox.from_json('{"title_text":"hello"}')

ValueError: Invalid JSON for this class

In [72]:
class ConfirmBox(MessageBox):
    def __init__(self, title_text, body_text,
                 dismiss_button='Cancel', confirm_button='Proceed'):
        super().__init__(title_text, body_text, dismiss_button)
        self.buttons.append(confirm_button)
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button', 'confirm_button'}
        return required <= set(data) <= everything

c = ConfirmBox.from_json('''
{
  "title_text": "Danger",
  "body_text": "Vent radioactive gas?",
  "confirm_button": "Yes I'm sure"
}''')
c

<ConfirmBox Danger>

In [73]:
c.buttons

['Cancel', "Yes I'm sure"]

In [75]:
ConfirmBox.valid_json(
    '{"title_text":"hi","body_text":"how are you?",'
    '"confirm_button":"good","dismiss_button":"bad"}')

True

## Descriptors
The power behind `property`, `classmethod`, `staticmethod` and normal functions becoming methods

In [111]:
class Tripwire:
    def __get__(self, obj, typ=None):
        print('caught access within', typ)
        if obj:
            print('from object', obj)
        return 'looks legit'

class Building:
    doorway = Tripwire()

b = Building()
b.doorway

caught access within <class '__main__.Building'>
from object <__main__.Building object at 0x7f9a82bb8668>


'looks legit'

In [112]:
Building.doorway

caught access within <class '__main__.Building'>


'looks legit'

In [86]:
def i_could_be_a_method(self):
    print('hello', self)

i_could_be_a_method.__get__

<method-wrapper '__get__' of function object at 0x7f9a82bca620>

In [89]:
m = i_could_be_a_method.__get__('fake')
m

<bound method i_could_be_a_method of 'fake'>

In [90]:
m()

hello fake


In [107]:
from itertools import count, repeat

class TicketDispenser:
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, typ=None):
        if not obj:
            raise AttributeError('dispenser requires object')
        c = obj.__dict__.setdefault(self.name, count(1))
        return next(c)

    def __set__(self, obj, value):
        obj.__dict__[self.name] = count(value)

    def __delete__(self, obj):
        obj.__dict__[self.name] = repeat('out of order')
        
class ServiceCounter:
    support = TicketDispenser('support')

sc1 = ServiceCounter()
sc2 = ServiceCounter()

print(sc1.support)
print(sc1.support)

1
2


In [108]:
print(sc2.support)
sc2.support = 500
print(sc2.support)
print(sc2.support)
print(sc1.support)
print(sc1.support)

1
500
501
3
4


In [109]:
del sc1.support
print(sc1.support)
print(sc1.support)

out of order
out of order


In [110]:
ServiceCounter.support

AttributeError: dispenser requires object

See also https://docs.python.org/3/howto/descriptor.html#descriptor-protocol

## Custom Containers

In [46]:
class BitList:
    """Mutable container of 0's and 1's"""
    value = 0

    @classmethod
    def from_int(cls, val):
        obj = cls()
        obj.value = int(val)
        return obj
        
    def __getitem__(self, bit):
        return (self.value >> bit) & 1
    
    def __setitem__(self, bit, val):
        if val != 0 and val != 1:
            raise ValueError('can only store 0 and 1')
        self.value = self.value & ~(1 << bit) | (val << bit)
    
bb = BitList.from_int(0x80ff)
bb[0]

1

In [47]:
bb[1000]

0

In [48]:
# WARNING would run forever:  list(bb)
# because __getitem__ never raises IndexError

In [49]:
def __len__(self):
    return self.value.bit_length()

BitList.__len__ = __len__
    
def __iter__(self):
    v = self.value
    while v:
        yield v & 1
        v >>= 1
        
BitList.__iter__ = __iter__


In [50]:
list(bb)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [51]:
list(reversed(bb))

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [52]:
bb[10] = 1
bb[1] = 0
list(bb)

[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [53]:
@classmethod
def from_bytes(cls, byt):
    obj = cls()
    obj.value = int.from_bytes(byt, 'little')
    return obj

BitList.from_bytes = from_bytes

def __bytes__(self):
    return self.value.to_bytes(
        (self.value.bit_length() + 7) // 8, 'little')

BitList.__bytes__ = __bytes__

bytes(bb)

b'\xfd\x84'

In [54]:
list(BitList.from_bytes(b'\xfd\x84'))

[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [55]:
bb[:5]

TypeError: unsupported operand type(s) for >>: 'int' and 'slice'

In [67]:
def __getitem__(self, bit_or_slice):
    if isinstance(bit_or_slice, slice):
        return [
            (self.value >> n) & 1 for n in
            range(*bit_or_slice.indices(len(self)))
        ]
    return (self.value >> bit_or_slice) & 1

BitList.__getitem__ = __getitem__

bb[:7]

[1, 0, 1, 1, 1, 1, 1]

In [62]:
bb[-5:]

[0, 0, 0, 0, 1]

In [63]:
bb[:]

[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [72]:
bb[...]

TypeError: unsupported operand type(s) for >>: 'int' and 'ellipsis'

In [73]:
bb[1,...]

TypeError: unsupported operand type(s) for >>: 'int' and 'tuple'

### 🏠🏠🏠 Idiomatic Python: Edge Cases, Duck Typing

- If the exception raised is the right type, consider not special casing every possible error case just to customize your exception messages
- Instead of `isinstance` use the values passed and catch exceptions

In [78]:
def __getitem__(self, index):
    if isinstance(index, slice):
        return [
            (self.value >> n) & 1 for n in
            range(*index.indices(len(self)))
        ]
    
    elif index == Ellipsis:
        return ''.join('🐔' if v else '🥚' for v in self)
    
    try:
        return (self.value >> index) & 1
    except TypeError as e:
        raise TypeError('BitList indices must be integers') from e

BitList.__getitem__ = __getitem__

bb[...]

'🐔🥚🐔🐔🐔🐔🐔🐔🥚🥚🐔🥚🥚🥚🥚🐔'

In [79]:
bb['key']

TypeError: BitList indices must be integers